In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [3]:
data = pd.read_csv('./redacted-Results-April21-Sept21.csv')
loc = pd.read_csv('liquor_location_1.csv').rename(columns = {'Zip Code' : 'zip', '??(????)' : 'latitude', '??(????).1' : 'longitude'})

In [4]:
data.head(5)

,id,notes,original_zip,manual_match_zip,match,match_kind,state_code,county_code,tract_code,block_code,...,Rental Relief Fund: How much do you owe on utilities listed?,Rental Relief Fund: Total Monthly Income all sources,Rental Relief Gap,Months of Assistance Awarded,Date of Final Decision,Utility Payments - Amount obtained ($),Future Stipend - Amount obtained ($),Rental Arrears - Amount obtained ($),Moving Costs - Amount obtained ($),Total RRF Award
0,161733,NaN,2121.0,NaN,Match,Non_Exact,25.0,25.0,10802.0,2001.0,...,$0.00,$0.00,$0.00,July-21; August-21; September-21; October-21; ...,NaN,$197.88,"$6,000.00","$2,000.00",NaN,"$8,197.88"
1,161811,NaN,2124.0,NaN,Match,Non_Exact,25.0,25.0,701.0,3010.0,...,$299.11,"$1,203.00","$3,499.11",21-Apr,2004/7/21,NaN,NaN,"$3,200.00",NaN,"$3,200.00"
2,161824,NaN,2126.0,NaN,Match,Non_Exact,25.0,25.0,92400.0,4001.0,...,"$1,724.21",$794.00,"$1,724.21",NaN,NaN,"$1,500.00",NaN,NaN,NaN,"$1,500.00"
3,161837,NaN,2128.0,NaN,Match,Non_Exact,25.0,25.0,10802.0,2003.0,...,$0.00,"$1,000.00","$30,600.00",May-21; June-21; July-21; August-21; September-21,NaN,NaN,NaN,"$15,000.00",NaN,"$15,000.00"
4,161839,NaN,2128.0,NaN,No_Match,NaN,NaN,NaN,50700.0,1001.0,...,$0.00,"$1,192.00","$1,501.00",September-21; October-21; November-21,8/17/21,NaN,"$4,500.00","$1,501.00",NaN,"$6,001.00"


In [5]:
def to_num(col):
    a = []
    x = col.to_numpy()
    for i in range(0, len(x)):
        t = str(x[i]).replace('$', '').replace(',', '')
        a.append(float(t))
    return a



In [6]:
data['Total RRF Award'] = to_num(data['Total RRF Award'])

In [7]:
data['Rental Relief Fund: How much rent do you owe due to COVID-19'] = to_num(data['Rental Relief Fund: How much rent do you owe due to COVID-19'].fillna(0))

In [8]:
data['Rental Relief Fund: How much do you owe on utilities listed?'] = to_num(data['Rental Relief Fund: How much do you owe on utilities listed?'].fillna(0))

In [9]:
data['total_owe'] = data['Rental Relief Fund: How much do you owe on utilities listed?']+data['Rental Relief Fund: How much rent do you owe due to COVID-19']

In [10]:
def zip(col):
    p = []
    a = col.to_numpy()
    for i in range(0, len(a)):
        x = '0'+str(a[i]).replace('.0', '')
        p.append(x)
    return p


In [11]:
data['original_zip'] = data['original_zip'].fillna('')
data['original_zip'] = zip(data['original_zip'])

In [12]:
data = data.rename(columns={'original_zip': 'zip'})
loc['zip'] = zip(loc['zip'])

In [13]:
data_loc = pd.merge(data, loc, how = 'left', on = 'zip')
data_loc = data_loc.fillna(0)
data_loc.head(1)

,id,notes,zip,manual_match_zip,match,match_kind,state_code,county_code,tract_code,block_code,...,Date of Final Decision,Utility Payments - Amount obtained ($),Future Stipend - Amount obtained ($),Rental Arrears - Amount obtained ($),Moving Costs - Amount obtained ($),Total RRF Award,total_owe,Counts,latitude,longitude
0,161733,0,02121,0,Match,Non_Exact,25.0,25.0,10802.0,2001.0,...,0,$197.88,"$6,000.00","$2,000.00",0,8197.88,0.0,8.0,42.3074,-71.0828


In [14]:
RRF = data_loc[['latitude', 'longitude', 'Total RRF Award']]
RRF_list = RRF.dropna().values.tolist()
m = folium.Map(location = [42.3601, -71.0589],
               zoom_start = 12)
HeatMap(RRF_list).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
m.save('RRF_SEP.html')

In [15]:
owe = data_loc[['latitude', 'longitude', 'total_owe']]
owe_list = owe.dropna().values.tolist()
m = folium.Map(location = [42.3601, -71.0589],
               zoom_start = 12)
HeatMap(owe_list).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
m.save('total_owed.html')

In [16]:
num_of_rrf_to_zip = data_loc.groupby('zip').count().reset_index()[['zip', 'Counts']].sort_values('Counts', ascending = False)
num_of_rrf_to_zip.to_excel('./num_of_rrf_to_zip_SEP.xlsx')

In [17]:
total_RRF_to_zip_SEP = data_loc.groupby('zip').sum().reset_index()[['zip', 'Total RRF Award']].sort_values('Total RRF Award', ascending = False)
total_RRF_to_zip_SEP.to_excel('./total_RRF_to_zip_SEP.xlsx')

In [18]:
total_owed_to_zip_SEP = data_loc.groupby('zip').sum().reset_index()[['zip', 'total_owe']].sort_values('total_owe', ascending = False)
total_owed_to_zip_SEP.to_excel('./total_owed_to_zip_SEP.xlsx')

In [28]:
pp = pd.merge(num_of_rrf_to_zip, total_owed_to_zip_SEP, on = 'zip')
pp['Average owe'] = pp['total_owe'] / pp['Counts']
pp.to_excel('./Average_owe_to_zip_SEP.xlsx')